# Read image and cut into 100 px x 100 px patches

In [1]:
import cv2

In [2]:
from sklearn.feature_extraction import image

In [3]:
img = cv2.imread("shadow_remove.jpg")
img

array([[[ 18,  56,  50],
        [  4,  42,  36],
        [  0,  32,  25],
        ...,
        [ 67, 182, 163],
        [ 34, 150, 143],
        [  8, 127, 126]],

       [[ 56, 100,  93],
        [ 42,  87,  78],
        [ 32,  73,  65],
        ...,
        [ 83, 185, 167],
        [ 53, 156, 149],
        [ 26, 128, 127]],

       [[ 49, 109,  98],
        [ 42, 100,  89],
        [ 39,  92,  82],
        ...,
        [106, 186, 169],
        [ 71, 145, 139],
        [ 27,  99,  99]],

       ...,

       [[156, 233, 205],
        [ 86, 171, 139],
        [ 70, 165, 131],
        ...,
        [ 35,  91,  68],
        [ 43, 100,  75],
        [ 51, 108,  83]],

       [[129, 216, 188],
        [ 78, 169, 136],
        [ 72, 167, 130],
        ...,
        [ 46,  97,  77],
        [ 38,  99,  73],
        [ 33, 100,  71]],

       [[ 30, 122,  93],
        [ 48, 142, 108],
        [ 67, 160, 121],
        ...,
        [ 48,  96,  78],
        [ 17,  82,  56],
        [  0,  72,  42]]

In [4]:
img.shape

(20000, 20000, 3)

In [ ]:
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
patches = image.extract_patches_2d(img, (100, 100), max_patches= 2000) #img is too large

C:\Anaconda\lib\site-packages\sklearn\feature_extraction\image.py:287: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  indexing_strides = arr[slices].strides


In [15]:
patches.shape

(2000, 100, 100, 3)

# Feed patches into pretrained ResNet deep network

In [23]:
import keras

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Clustering

In [9]:
from sklearn.cluster import KMeans

In [16]:
kmeans = KMeans(n_clusters=10)
kmeans.fit(patches)
y_kmeans = kmeans.predict(patches)

ValueError: Found array with dim 4. Estimator expected <= 2.

In [18]:
cv2.kmeans(patches, 5)

TypeError: Required argument 'bestLabels' (pos 3) not found